## Run/ check Elastic search

    systemctl start elasticsearch

In [14]:
import requests
res = requests.get('http://localhost:9200')
print(res.content)

{
  "name" : "u4iVhDd",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "biTULKRYTPKW62ewYH9Q9g",
  "version" : {
    "number" : "6.4.2",
    "build_flavor" : "default",
    "build_type" : "deb",
    "build_hash" : "04711c2",
    "build_date" : "2018-09-26T13:34:09.098244Z",
    "build_snapshot" : false,
    "lucene_version" : "7.4.0",
    "minimum_wire_compatibility_version" : "5.6.0",
    "minimum_index_compatibility_version" : "5.0.0"
  },
  "tagline" : "You Know, for Search"
}



## import ElasticSearch

In [15]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

## read dataframes and import data to ES

In [16]:
import pandas as pd

In [17]:
oracle = pd.read_csv('oracleTM.csv', index_col=0)
oracle.drop_duplicates(inplace=True)

In [18]:
wikibooks = pd.read_csv('data.csv', index_col=0)
wikibooks.drop_duplicates(inplace=True)

In [19]:
# df = oracle.append(wikibooks)
# df = wikibooks

## apply text pre-processing and create a dunction to apply for test data
    lowercase, stemming, stop-word removal

In [20]:
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer
stop = text.ENGLISH_STOP_WORDS
porter_stemmer = PorterStemmer()
# df['text'] = df["text"].apply(lambda x: ''.join(["" if ord(i) < 32 or ord(i) > 126 else i for i in x]))

wikibooks['text'] = wikibooks["text"].apply(lambda x: ''.join(["" if ord(i) < 32 or ord(i) > 126 else i for i in x]))

oracle['text'] = oracle["text"].apply(lambda x: ''.join(["" if ord(i) < 32 or ord(i) > 126 else i for i in x]))

In [21]:
# df['text'] = df['text'].apply(lambda x: ' '.join([porter_stemmer.stem(word.lower()) for word in x.split() if word not in (stop)]))

wikibooks['text'] = wikibooks['text'].apply(lambda x: ' '.join([porter_stemmer.stem(word.lower()) for word in x.split() if word not in (stop)]))

oracle['text'] = oracle['text'].apply(lambda x: ' '.join([porter_stemmer.stem(word.lower()) for word in x.split() if word not in (stop)]))

In [22]:
# df

In [23]:
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer
    
def preprocessing(df_col):
    stop = text.ENGLISH_STOP_WORDS
    porter_stemmer = PorterStemmer()
    df_col = df_col.apply(lambda x: ''.join(["" if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    df_col = df_col.apply(lambda x: ' '.join([porter_stemmer.stem(word.lower()) for word in x.split() if word not in (stop)]))
    return df_col

## insert records to elastic search

In [24]:
# df['json'] = df.apply(lambda x: x.to_json(), axis=1)

wikibooks['json'] = wikibooks.apply(lambda x: x.to_json(), axis=1)

oracle['json'] = oracle.apply(lambda x: x.to_json(), axis=1)

In [26]:
es.indices.delete(index='java_oracle', ignore=[400, 404])
#for index in range(len(oracle)):
#    es.index(index='java_oracle', doc_type='document', id=index, body=oracle.iloc[index]['json'])

es.indices.delete(index='java_wikibook', ignore=[400, 404])

for index in range(len(wikibooks)):
    es.index(index='java_wikibook', doc_type='document', id=index, body=wikibooks.iloc[index]['json'])

"""
es.indices.delete(index='java', ignore=[400, 404])
for index in range(len(df)):
    es.index(index='java', doc_type='document', id=index, body=df.iloc[index]['json'])
    
for index in range(len(oracle)):
    es.index(index='java_oracle', doc_type='document', id=index, body=oracle.iloc[index]['json'])
"""

"\nes.indices.delete(index='java', ignore=[400, 404])\nfor index in range(len(df)):\n    es.index(index='java', doc_type='document', id=index, body=df.iloc[index]['json'])\n    \nfor index in range(len(oracle)):\n    es.index(index='java_oracle', doc_type='document', id=index, body=oracle.iloc[index]['json'])\n"

## read queries

In [ ]:
queries = pd.read_excel('queries.xlsx')

In [ ]:
queries['text_1'] = preprocessing(queries['text'])

In [ ]:
import collections 
ret = []

for index in range(len(queries)):
    #query = queries.iloc[index]['text_1']
    query = queries.iloc[index]['text']
    query_text = queries.iloc[index]['text']
    
    res_wikibooks = es.search(index='java_wikibook',from_=0, size=10, body={"query": {
        "multi_match" : {
          "query":    query,
          "fields": [ "title"] 
        }
    }})
    
    result = {}
    result['data'] = []
    
    for item in res_wikibooks['hits']['hits']:
        d = {}
        title = item['_source']['title']
        url = item['_source']['url']
        d['title'] = title
        d['url'] = url
        result['data'].append(d)
        result['title'] = query_text
    ret.append(result)

In [ ]:
ret

In [ ]:
import json
with open('file.json', 'w') as outfile:
    json.dump(ret, outfile)

In [ ]:
import collections 
import pandas as pd
ret = []

queries = pd.read_excel('queries.xlsx')

for index in range(len(queries)):
    query = queries.iloc[index]['text']
    query_text = queries.iloc[index]['text']
    
    res_wikibooks = es.search(index='java_wikibook',from_=0, size=10, body={"query": {
        "multi_match" : {
          "query":    query,
          "fields": [ "title"] 
        }
    }})
    
    result = {}
    result['data'] = []
    
    for item in res_wikibooks['hits']['hits']:
        d = {}
        title = item['_source']['title']
        url = item['_source']['url']
        d['title'] = title
        d['url'] = url
        result['data'].append(d)
        result['title'] = query_text
    ret.append(result)
import json
with open('file.json', 'w') as outfile:
    json.dump(ret, outfile)